## CH02

In [5]:
! conda install anaconda::scikit-learn -y

Channels:
 - defaults
 - anaconda
 - conda-forge
 - pytorch
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\user\anaconda3\envs\torch-book

  added / updated specs:
    - anaconda::scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    icc_rt-2022.1.0            |       h6049295_2         6.5 MB
    joblib-1.4.2               |  py312haa95532_0         515 KB
    pybind11-abi-5             |       hd3eb1b0_0          14 KB
    scikit-learn-1.3.0         |  py312hc7c4135_2         9.4 MB  anaconda
    scipy-1.13.1               |  py312hbb039d4_0        22.5 MB
    threadpoolctl-3.5.0        |  py312hfc267ef_0          49 KB
    ------------------------------------------------------------
                                           Total:        39.0 MB

The following NEW packages will be INSTALLED:

  icc_rt           

In [1]:
import torch

In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
dataset = pd.read_csv("data/car_evaluation.csv")

In [4]:
dataset.head()

,price,maint,doors,persons,lug_capacity,safety,output
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [6]:
categorical_columns = ["price", "maint", "doors", "persons", "lug_capacity", "safety"]

In [ ]:
for category in categorical_columns:
    dataset[category] = dataset[category].astype("category")

In [ ]:
price = dataset["price"].cat.codes.values
maint = dataset["maint"].cat.codes.values
doors = dataset["doors"].cat.codes.values
persons = dataset["persons"].cat.codes.values
lug_capacity = dataset["lug_capacity"].cat.codes.values
safety = dataset["safety"].cat.codes.values

categorical_data = np.stack([price, maint, doors, persons, lug_capacity, safety], 1)
categorical_data[:10]

In [ ]:
categorical_data = torch.tensor(categorical_data, dtype=torch.int64)
categorical_data[:10]

In [ ]:
outputs = pd.get_dummies(dataset.output)
outputs = outputs.values
outputs = torch.tensor(outputs).flatten()

print(categorical_data.shape)
print(outputs.shape)

In [ ]:
categorical_column_sizes = [
    len(dataset[column].cat.categories) for column in categorical_columns
]


categorical_embedding_sizes = [
    (col_size, min(50, (col_size + 1) // 2)) for col_size in categorical_column_sizes
]


print(categorical_embedding_sizes)

In [ ]:
total_records = 1728
test_records = int(total_records * 0.2)

categorical_train_data = categorical_data[: total_records - test_records]
categorical_test_data = categorical_data[total_records - test_records : total_records]
train_outputs = outputs[: total_records - test_records]
test_outputs = outputs[total_records - test_records : total_records]

In [ ]:
print(len(categorical_train_data))
print(len(train_outputs))
print(len(categorical_test_data))
print(len(test_outputs))

In [ ]:
class Model(nn.Module):
    def __init__(self, embedding_size, output_size, layers, p=0.4):
        super().__init__()
        self.all_embeddings = nn.ModuleList(
            [nn.Embedding(ni, nf) for ni, nf in embedding_size]
        )
        self.embedding_dropout = nn.Dropout(p)
        all_layers = []
        num_categorical_cols = sum((nf for ni, nf in embedding_size))
        input_size = num_categorical_cols
        for i in layers:
            all_layers.append(nn.Linear(input_size, i))
            all_layers.append(nn.ReLU(inplace=True))
            all_layers.append(nn.BatchNorm1d(i))
            all_layers.append(nn.Dropout(p))
            input_size = i
        all_layers.append(nn.Linear(layers[-1], output_size))
        self.layers = nn.Sequential(*all_layers)

    def forward(self, x_categorical):
        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        x = torch.cat(embeddings, 1)
        x = self.embedding_dropout(x)
        x = self.layers(x)
        return x

In [ ]:
model = Model(categorical_embedding_sizes, 4, [200, 100, 50], p=0.4)
print(model)

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
epochs = 500
aggregated_losses = []
train_outputs = train_outputs.to(device="cpu", dtype=torch.int64)
for i in range(epochs):
    i += 1
    y_pred = model(categorical_train_data)
    single_loss = loss_function(y_pred, train_outputs)
    aggregated_losses.append(single_loss)

    if i % 25 == 1:
        print(f"epoch: {i:3} loss: {single_loss.item():10.8f}")

    optimizer.zero_grad()
    single_loss.backward()
    optimizer.step()

print(f"epoch: {i:3} loss: {single_loss.item():10.10f}")

In [ ]:
test_outputs = test_outputs.to(device=device, dtype=torch.int64)
with torch.no_grad():
    y_val = model(categorical_test_data)
    loss = loss_function(y_val, test_outputs)
print(f"Loss: {loss:.8f}")

In [ ]:
print(y_val[:5])

In [ ]:
y_val = np.argmax(y_val, axis=1)
print(y_val[:5])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(test_outputs, y_val))
print(classification_report(test_outputs, y_val))
print(accuracy_score(test_outputs, y_val))